In [1]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import math
import random
from sklearn.metrics import f1_score, roc_auc_score, confusion_matrix
import time
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import dgl.function as fn
from dgl.nn.pytorch import GraphConv, SAGEConv

Using backend: pytorch


In [3]:
from deepsurv_utils import c_index, adjust_learning_rate
from loss import NegativeLogLikelihood

In [4]:
# load data
all_patient_info = pd.read_csv("/home/jielian/lung-graph-project/data/csv/SPH0812.csv")
stage1 = list(np.load("/home/jielian/lung-graph-project/data/seg_image/labels/name_stage1.npy"))
stage2 = list(np.load("/home/jielian/lung-graph-project/data/seg_image/labels/name_stage2.npy"))
patint_list = [*stage1, *stage2]
patient_info = all_patient_info[all_patient_info['folder_name'].isin(patint_list)]
feature_files = os.listdir("trans_feature")

In [5]:
data = []
name = []
for feature_name in feature_files:
    path = "trans_feature/"+feature_name
    name.append(int(feature_name[:-4]))
    feature = list(np.load(path, allow_pickle=True))
    data.append(feature)
feature_data = pd.DataFrame(data)
feature_data['folder_name']=name
all_data = patient_info.merge(feature_data, how='left', on='folder_name')

In [6]:
train_id = np.load("data_ind/train_index.npy",allow_pickle=True)
val_id = np.load("data_ind/val_index.npy", allow_pickle=True)
test_id = np.load("data_ind/test_index.npy",allow_pickle=True)
idx_train = torch.LongTensor(train_id)
idx_val = torch.LongTensor(val_id)
idx_test = torch.LongTensor(test_id)

print("training survival distribution:")
print(all_data.iloc[train_id,:]['OS_Status'].value_counts())
print("validation survival distribution:")
print(all_data.iloc[val_id,:]['OS_Status'].value_counts())
print("test survival distribution:")
print(all_data.iloc[test_id,:]['OS_Status'].value_counts())


# print("training survival distribution:")
# print(all_data.iloc[train_id,:]['RFS_Status'].value_counts())
# print("validation survival distribution:")
# print(all_data.iloc[val_id,:]['RFS_Status'].value_counts())
# print("test survival distribution:")
# print(all_data.iloc[test_id,:]['RFS_Status'].value_counts())

training survival distribution:
0    991
1    287
Name: OS_Status, dtype: int64
validation survival distribution:
0    173
1     41
Name: OS_Status, dtype: int64
test survival distribution:
0    169
1     44
Name: OS_Status, dtype: int64


In [9]:
import scipy.stats as st
data = all_data['RFS_Status'].to_list()
print(st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data)) )

(0.24984968029262525, 0.2920858035783425)


In [31]:
ttt = [*train_id, *val_id]

In [32]:
all_data.iloc[ttt,:]['Location_1_LUL_2_LLL_3_RUL_4_RML_5_RLL'].value_counts()

3      488
1      370
5      247
2      211
4       99
45      47
12      14
34      13
345      2
35       1
Name: Location_1_LUL_2_LLL_3_RUL_4_RML_5_RLL, dtype: int64

In [46]:
247/(384+211+504+146+247)

0.16554959785522788

In [7]:
# define similarity of two patient
def SimScore(a1,a2,s1,s2,l1,l2,h1,h2,t1,t2,n1,n2,m1,m2,tnm1,tnm2): 
    c_score = 0
    h_score = 0
    t_score = 0
    # sex and age
    if s1 == s2:
        c_score +=1
    if abs(a1-a2) <= 5:
        c_score +=1
    
    if l1 == l2:
        h_score +=1
    if h1 == h2:
        h_score +=1
    
    if t1 == t2:
        t_score +=1
    if n1 == n2:
        t_score +=1
    if m1 == m2:
        t_score +=1
#     if tnm1 == tnm2:
#         t_score +=1

    return c_score*t_score*h_score

# def SimScore(a1,a2,s1,s2,l1,l2,h1,h2,t1,t2,n1,n2,m1,m2,tnm1,tnm2): 

#     return c_score*t_score*h_score


def adj_matrix(patient_info):
    age = patient_info['Age'].to_list()
    sex = patient_info['Sex_1_male_2_female'].to_list()
    
    loc = patient_info['Location_1_LUL_2_LLL_3_RUL_4_RML_5_RLL'].to_list()
    his = patient_info['Histology_1_Adenocarcinoma_2_SquamousCellCarcinoma_3_Others'].to_list()
    pts = patient_info['pT_Stage'].to_list()
    pns = patient_info['pN_Stage'].to_list()
    pms = patient_info['pM_Stage'].to_list()
    tnm = patient_info['pTNM'].to_list()

    edge_list=[]
    edge_wight=[]
    n_sample = len(age)
    adj = np.zeros((n_sample, n_sample))
    for i in tqdm(range(n_sample)):
        for j in range(n_sample):
            adj[i,j] = SimScore(age[i],age[j],sex[i],sex[j],loc[i],loc[j],his[i],his[j],
                                pts[i],pts[j],pns[i],pns[j], pms[i],pms[j],tnm[i],tnm[j])
            if adj[i,j] != 0:
                edge_list.append([i,j])
                edge_wight.append(adj[i,j])
    return adj, edge_list,edge_wight

In [8]:
adj, edge_list, edge_wight = adj_matrix(all_data)
print("the number of edges in this graph:",len(edge_list))
print("Number of average degree: ",len(edge_list)/1705 )

100%|██████████████████████████████████████| 1705/1705 [00:04<00:00, 366.75it/s]

the number of edges in this graph: 1342873
Number of average degree:  787.608797653959


In [9]:
# save the labels
norm_label = all_data['OS_Month']
# norm_label = (all_data['OS_Month']-np.min(all_data['OS_Month']))/(np.max(all_data['OS_Month'])-np.min(all_data['OS_Month']))
labels = torch.from_numpy(norm_label.to_numpy())
events = torch.from_numpy(all_data['OS_Status'].to_numpy())
# build graph struture data
g = dgl.DGLGraph()
g.add_nodes(len(labels))
# add nodes
# node_feature = (all_data.iloc[:, 15:]-all_data.iloc[:, 15:].min())/(all_data.iloc[:, 15:].max()- all_data.iloc[:, 15:].min())
node_feature = all_data.iloc[:, 15:]
# print(node_feature)

/home/jielian/anaconda3/envs/gnn/lib/python3.8/site-packages/dgl/base.py:45: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  return warnings.warn(message, category=category, stacklevel=1)


In [10]:
node_feature_norm = node_feature.to_numpy()
g.ndata['h'] = torch.from_numpy(node_feature_norm).float()
g.ndata['event'] = events
g.ndata['label'] = labels
g.ndata
# g.adj = adj
# add edges
src, dst = tuple(zip(*edge_list))
g.add_edges(src, dst)
# add edge weight
edge_wight = np.array(edge_wight)
g.edata['w'] = torch.from_numpy(edge_wight).float()

In [11]:
class SAGE(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, dropout=0, activation = None,aggregator_type='mean'):
        super().__init__()
        self.fc1 = nn.Linear(in_feats, hid_feats) 
        self.conv1 = SAGEConv(in_feats=hid_feats, out_feats=64, aggregator_type=aggregator_type, activation=activation, feat_drop=dropout)
        self.conv2 = SAGEConv(in_feats=64, out_feats= out_feats, aggregator_type=aggregator_type, activation=activation, feat_drop=dropout)
        self.fc2 = nn.Linear(out_feats, 1) 
    def forward(self, graph, inputs, w_input):
        # inputs are features of nodes
#         graph.ndata['h']= self.fc1(inputs)
        h = self.fc1(inputs)
        if w_input != None:
            h = self.conv1(graph, h, w_input)
        else:
            h = self.conv1(graph, h)
        h = self.conv2(graph,h)
#         print(h.size())
#         output=F.relu(self.fc2(h))
        output=self.fc2(h)

        return output

In [12]:
class GCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, activation = F.softmax, norm ="both"):
        super().__init__()
        self.fc1 = nn.Linear(in_feats, hid_feats) 
        self.conv1 = GraphConv(in_feats=hid_feats, out_feats= hid_feats, activation=activation, norm=norm)
        self.conv2 = GraphConv(in_feats=hid_feats, out_feats= out_feats,  activation=activation, norm=norm)
        self.fc2 = nn.Linear(out_feats, 1) 
        
    def forward(self, graph, inputs, w_input):
        # inputs are features of nodes
        h= self.fc1(inputs)
        h = self.conv1(graph, h)
        h = self.conv2(graph,h)
#         output=F.relu(self.fc2(h))
        h=self.fc2(h)
        
        return h

In [13]:
def train(g, model, save_dic, idx_train,idx_val, idx_test, total_epoch=100, patience=5, lr=0.001, reg_l2=0, weight_decay=0.0001):
    model_name = save_dic['model']+str(save_dic['hid_feats'])+str(save_dic['out_feats'])+str(save_dic['reg_l2'])+save_dic["aggregator_type"]
    optimizer = torch.optim.Adam(model.parameters(),lr=lr, weight_decay=weight_decay)
    best_cindex = 0
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',factor=0.5, patience=patience, min_lr = 0.0001, verbose=True)
    criterion = NegativeLogLikelihood(reg_l2)
    features = g.ndata['h']
#     e_feature = g.edata['w']
    e_feature = None
    labels = g.ndata['label']
    events = g.ndata['event']
    t_total = time.time()
    with tqdm(range(total_epoch)) as t:
        for epoch in t:
            t.set_description('Epoch %d' % epoch)
            start = time.time()
            model.train()
            optimizer.zero_grad()
            output = model(g, features,e_feature)
            # Compute loss
            # Note that you should only compute the losses of the nodes in the training set.
            loss_train = criterion(output[idx_train], labels[idx_train],events[idx_train], model).clone()
            auc_train = c_index(-output[idx_train], labels[idx_train],events[idx_train])
            
            loss_train.backward(retain_graph=True)
            optimizer.step()
            
            model.eval()
            val_output = model(g, features,e_feature)
            loss_val = criterion(val_output[idx_val], labels[idx_val],events[idx_val], model).clone()
            scheduler.step(loss_val)
            
            auc_val = c_index(-val_output[idx_val], labels[idx_val],events[idx_val])
            auc_test = c_index(-val_output[idx_test], labels[idx_test],events[idx_test])

            if auc_test>best_cindex:
                best_cindex = auc_test.item()
                print("Curent best :", auc_test)
                print("Its Val ACU:",auc_val)
                if epoch > 0 and best_cindex > 0.7 and auc_val > 0.65:
                    torch.save(model.state_dict(), os.path.join(save_dic['save_path'], 
                                "{}_ep{}_val{}_test{}.pth.gz".format(model_name,epoch, np.around(auc_val,3),np.around(auc_test,3))))

            t.set_postfix(
                  {"train_loss":loss_train.item(), "val_loss":loss_val.item(),
                  "train_cindex":auc_train.item(), "val_auc":auc_val.item(),
                "lr":optimizer.param_groups[0]['lr']}) 

In [20]:
save_dic = {"model":"SAGE", 
            "hid_feats":96, 
            'out_feats':16, 
            'reg_l2':0,
            "aggregator_type":'mean',
            "save_path":"/home/jielian/lung-graph-project/Tumor_tranformer/logs/models/"}
sage96_norm = SAGE(g.ndata['h'].shape[1],hid_feats=save_dic["hid_feats"],out_feats=save_dic['out_feats'], 
                   activation = F.softmax, aggregator_type=save_dic['aggregator_type'])
train(g, sage96_norm, save_dic, idx_train,idx_val, idx_test, 50, patience=5, reg_l2=save_dic["reg_l2"])

Epoch 3:   6%| | 3/50 [00:00<00:04, 11.50it/s, train_loss=8.97e-5, val_loss=-2.1

Curent best : 0.6227927363807139
Its Val ACU: 0.5309229305423406


Epoch 11:  22%|▏| 11/50 [00:00<00:02, 15.66it/s, train_loss=0.000263, val_loss=-

Epoch     8: reducing learning rate of group 0 to 5.0000e-04.


Epoch 20:  38%|▍| 19/50 [00:01<00:01, 16.27it/s, train_loss=0.000101, val_loss=3

...

Epoch 26:  50%|▌| 25/50 [00:01<00:01, 16.55it/s, train_loss=-.000177, val_loss=-

Epoch    23: reducing learning rate of group 0 to 1.2500e-04.


Epoch 32:  62%|▌| 31/50 [00:02<00:01, 16.81it/s, train_loss=-1.17e-5, val_loss=6

Epoch    29: reducing learning rate of group 0 to 1.0000e-04.


Epoch 49: 100%|█| 50/50 [00:03<00:00, 16.11it/s, train_loss=1.98e-5, val_loss=-2


In [17]:
save_dic = {"model":"SAGE", 
            "hid_feats":128, 
            'out_feats':16, 
            'reg_l2':0.00001,
            "aggregator_type":'mean',
            "save_path":"/home/jielian/lung-graph-project/Tumor_tranformer/logs/models/"}
sage96_norm = SAGE(g.ndata['h'].shape[1],hid_feats=save_dic["hid_feats"],out_feats=save_dic['out_feats'], 
                   activation = F.relu, aggregator_type=save_dic['aggregator_type'])
train(g, sage96_norm, save_dic, idx_train,idx_val, idx_test, 50, patience=5, reg_l2=save_dic["reg_l2"])

Epoch 3:   6%| | 3/50 [00:00<00:04, 11.02it/s, train_loss=7.81, val_loss=1.92, t

Curent best : 0.4791484032561052
Its Val ACU: 0.3886094875628653
Curent best : 0.6641202254226675
Its Val ACU: 0.6248470844094061


Epoch 10:  18%|▏| 9/50 [00:00<00:02, 15.29it/s, train_loss=0.721, val_loss=0.202

Epoch     7: reducing learning rate of group 0 to 5.0000e-04.


Epoch 31:  62%|▌| 31/50 [00:01<00:01, 16.33it/s, train_loss=0.21, val_loss=0.018

Curent best : 0.667501565435191
Its Val ACU: 0.6629060758461329
Curent best : 0.6687539135879774
Its Val ACU: 0.6537991028952018


Epoch 41:  82%|▊| 41/50 [00:02<00:00, 16.61it/s, train_loss=0.136, val_loss=0.02

Epoch    38: reducing learning rate of group 0 to 2.5000e-04.


Epoch 49: 100%|█| 50/50 [00:03<00:00, 16.12it/s, train_loss=0.0879, val_loss=0.0

Epoch    50: reducing learning rate of group 0 to 1.2500e-04.


In [19]:
save_dic = {"model":"GCN", 
            "hid_feats":128, 
            'out_feats':12, 
            'reg_l2':0.00001,
            "aggregator_type":'mean',
            "save_path":"/home/jielian/lung-graph-project/Tumor_tranformer/logs/models/"}
gcn_norm = GCN(g.ndata['h'].shape[1],hid_feats=save_dic["hid_feats"],out_feats=save_dic['out_feats'], 
                   activation = F.relu)
train(g, gcn_norm, save_dic, idx_train,idx_val, idx_test, 50, patience=5, reg_l2=save_dic["reg_l2"])

Epoch 2:   2%| | 1/50 [00:00<00:06,  7.94it/s, train_loss=0.0135, val_loss=0.715

Curent best : 0.2954289292423294
Its Val ACU: 0.336006524398532
Curent best : 0.6882905447714465
Its Val ACU: 0.6414299306782656


Epoch 24:  46%|▍| 23/50 [00:01<00:01, 14.11it/s, train_loss=-.000265, val_loss=0

Epoch    21: reducing learning rate of group 0 to 5.0000e-04.


Epoch 30:  58%|▌| 29/50 [00:02<00:01, 15.02it/s, train_loss=2.06e-5, val_loss=0.

...

Epoch 35:  70%|▋| 35/50 [00:02<00:00, 15.40it/s, train_loss=4.87e-5, val_loss=0.

Epoch    33: reducing learning rate of group 0 to 1.2500e-04.


Epoch 41:  82%|▊| 41/50 [00:03<00:00, 13.94it/s, train_loss=0.000144, val_loss=0

Epoch    39: reducing learning rate of group 0 to 1.0000e-04.


Epoch 49: 100%|█| 50/50 [00:03<00:00, 13.64it/s, train_loss=0.000344, val_loss=0


In [18]:
# #test!
# save_dic = {"model":"SAGE", 
#             "hid_feats":96, 
#             'out_feats':32, 
#             "aggregator_type":'mean',
#             "save_path":"/home/jielian/lung-graph-project/Tumor_tranformer/logs/models/"}
# features = g.ndata['h']
# e_feature = g.edata['w']
# labels = g.ndata['label']
# events = g.ndata['event']
# # model = SAGE(g.ndata['h'].shape[1],hid_feats=96,out_feats=32, activation = F.relu, aggregator_type="mean")
# model = SAGE(g.ndata['h'].shape[1],hid_feats=save_dic["hid_feats"],out_feats=save_dic['out_feats'], 
#                    activation = F.relu, aggregator_type=save_dic['aggregator_type'])
# pre_train='SAGE96321e-05mean_ep3_val0.646_test0.775.pth.gz'
# state_dict=torch.load(os.path.join(save_dic["save_path"]+pre_train), map_location='cpu')
# model.load_state_dict(state_dict)
# model.eval()
# outputs = model.forward(g, features,e_feature)
# auc_train = c_index(-outputs[idx_train], labels[idx_train],events[idx_train])
# print(auc_train)
# auc_val = c_index(-outputs[idx_val], labels[idx_val],events[idx_val])
# print(auc_val)
# auc_test = c_index(-outputs[idx_test], labels[idx_test],events[idx_test])
# print(auc_test)

In [ ]:
#test!
save_dic = {"model":"SAGE", 
            "hid_feats":128, 
            'out_feats':32, 
            "aggregator_type":'mean',
            "save_path":"/home/jielian/lung-graph-project/Tumor_tranformer/logs/models/"}
features = g.ndata['h']
e_feature = g.edata['w']
labels = g.ndata['label']
events = g.ndata['event']
model = SAGE(g.ndata['h'].shape[1],hid_feats=save_dic["hid_feats"],out_feats=save_dic['out_feats'], 
                   activation = F.relu, aggregator_type=save_dic['aggregator_type'])
pre_train='SAGE128320.1mean_ep3_val0.761_test0.75.pth.gz'
state_dict=torch.load(os.path.join(save_dic["save_path"]+pre_train), map_location='cpu')
model.load_state_dict(state_dict)
model.eval()
outputs = model.forward(g, features,e_feature)
auc_train = c_index(-outputs[idx_train], labels[idx_train],events[idx_train])
print(auc_train)
auc_val = c_index(-outputs[idx_val], labels[idx_val],events[idx_val])
print(auc_val)
auc_test = c_index(-outputs[idx_test], labels[idx_test],events[idx_test])
print(auc_test)